In [1]:
import csv
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from folium.plugins import BeautifyIcon

In [2]:
data = pd.read_csv("../data/rawYearlyData/clean_files/clean_2023.csv")
data.head()

,Unnamed: 0,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,0,2023,9,9E,Endeavor Air Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2062.0,200.0,66.78,...,0.0,83.45,2.0,1.0,17361.0,6394.0,4028.0,1503.0,0.0,5436.0
1,1,2023,9,9E,Endeavor Air Inc.,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",75.0,15.0,4.95,...,0.0,4.59,8.0,0.0,1059.0,265.0,0.0,273.0,0.0,521.0
2,2,2023,9,9E,Endeavor Air Inc.,CLT,"Charlotte, NC: Charlotte Douglas International",206.0,27.0,8.91,...,0.0,9.80,23.0,0.0,2430.0,1334.0,0.0,502.0,0.0,594.0
3,3,2023,9,9E,Endeavor Air Inc.,DCA,"Washington, DC: Ronald Reagan Washington National",167.0,29.0,8.22,...,0.0,7.06,2.0,0.0,1607.0,606.0,0.0,689.0,0.0,312.0
4,4,2023,9,9E,Endeavor Air Inc.,DTW,"Detroit, MI: Detroit Metro Wayne County",1612.0,215.0,56.10,...,0.0,105.60,19.0,2.0,17140.0,4703.0,1784.0,2340.0,0.0,8313.0


In [3]:
map_df = data.drop(columns=["Unnamed: 0", "carrier_ct", "carrier", "carrier_name", "weather_ct", "nas_ct","security_ct","late_aircraft_ct", "arr_diverted", "arr_delay","carrier_delay", "weather_delay","nas_delay", "security_delay", "late_aircraft_delay"])
map_df.head()

,year,month,airport,airport_name,arr_flights,arr_del15,arr_cancelled
0,2023,9,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2062.0,200.0,2.0
1,2023,9,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",75.0,15.0,8.0
2,2023,9,CLT,"Charlotte, NC: Charlotte Douglas International",206.0,27.0,23.0
3,2023,9,DCA,"Washington, DC: Ronald Reagan Washington National",167.0,29.0,2.0
4,2023,9,DTW,"Detroit, MI: Detroit Metro Wayne County",1612.0,215.0,19.0


In [4]:
airports_df = pd.read_csv("../data/top25_airport_codes.csv")
airports_table = airports_df.drop(columns=['Unnamed: 0'])
display(airports_table)

,country_code,iata,airport,latitude,longitude
0,US,SLC,Salt Lake City International Airport,40.7884,-111.9780
1,US,ATL,Hartsfield-Jackson Atlanta International Airport,33.6367,-84.4281
2,US,FLL,Fort Lauderdale-Hollywood International Airport,26.0726,-80.1527
3,US,MIA,Miami International Airport,25.7932,-80.2906
4,US,MCO,Orlando International Airport,28.4294,-81.3090
5,US,MSP,Minneapolis-Saint Paul International Airport (...,44.8820,-93.2218
6,US,LAX,Los Angeles International Airport,33.9425,-118.4080
7,US,LAS,Harry Reid International Airport,36.0801,-115.1520
8,US,DFW,Dallas/Fort Worth International Airport,32.8968,-97.0380
9,US,DEN,Denver International Airport,39.8617,-104.6730


In [5]:
m_airports = folium.Map(location=(39.8283, -98.5795),tiles="Cartodb dark_matter", zoom_start=4)

In [6]:
# calculate totals per airport
arr_airports = map_df.groupby('airport')["arr_flights"].sum()
delay_airports = map_df.groupby('airport')['arr_del15'].sum()
cancel_airports = map_df.groupby('airport')['arr_cancelled'].sum()

#calculate percentages
delay_percents = round((delay_airports/arr_airports)*100, 3)
cancel_percents = round((cancel_airports/arr_airports)*100, 3)

#gather latitude and longitue
airport_lat = airports_table['latitude']
airport_lng = airports_table['longitude']
airport_iata = airports_table['airport']

In [7]:
# Convert the DataFrame columns to native Python lists and types
lat = airport_lat.astype(float).tolist()
lng = airport_lng.astype(float).tolist()

delay = delay_percents.astype(float).tolist()
cancel = cancel_percents.astype(float).tolist()

# Zip the lists for folium format
delay_heat = list(zip(lat, lng, delay))
cancel_heat = list(zip(lat, lng, cancel))

In [11]:
# Create map features
marker_cluster = MarkerCluster(name="Airports").add_to(m_airports)

for row in airports_table.itertuples():
    folium.Marker(
        location=[row.latitude, row.longitude],
        popup=(f"<b>{row.iata}</b>: {row.airport}"),
        icon=folium.Icon(color="blue", icon="plane")
    ).add_to(marker_cluster)

# add delays to layer control
fg1 = folium.FeatureGroup(name="Delays", control=True).add_to(m_airports)
HeatMap(delay_heat, gradient={0.2:'blue', 0.4:'green', 0.6:'yellow', 0.8:'orange', 1: 'red'}).add_to(fg1)

# add cancellations to layer control
fg2 = folium.FeatureGroup(name="Cancellations", control=True).add_to(m_airports)
HeatMap(cancel_heat, gradient={0.2:'blue', 0.4:'green', 0.6:'yellow', 0.8:'orange', 1: 'red'}).add_to(fg2)

# add layer control to the map
folium.LayerControl(collapsed=False).add_to(m_airports)
m_airports